In [1]:
import torch
import numpy as np
from scipy.io import loadmat
import pandas as pd
import matplotlib.pyplot as plt
from scipy import signal


In [2]:
from torch.utils.data import TensorDataset, DataLoader

In [3]:
data = loadmat("exampleEMGdata180trial_train.mat")
trials = data['dataChTimeTr']
labels = data['labels'].squeeze().tolist()
labels = [n-1 for n in labels]
sr = data['actualFs']

In [4]:
images = []
for i in range(180):
    time_domain_data = trials[:,:,i]
    spectrograms = []
    for j in range(4):
        f, t, Sxx = signal.spectrogram(time_domain_data[j], fs= sr)
        spectrograms.append(Sxx)
    multichannel_single_trial = np.stack(spectrograms, axis=-1)
    images.append(multichannel_single_trial)
images = np.array(images)

In [5]:
images_torch = torch.tensor(images.transpose(0,3,1,2), dtype=torch.float32)
dataset = TensorDataset(images_torch, torch.tensor(labels,dtype=torch.long))
loader = DataLoader(dataset, batch_size=32, shuffle=True)

In [6]:
import torch.nn as nn
import torch.optim as optim

In [7]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [40]:
class MultiChannelCNN(nn.Module):
    def __init__(self):
        super(MultiChannelCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=4, out_channels=32, kernel_size=(20,2), padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=(10,2), stride=2)

        self.conv2 = nn.Conv2d(in_channels=32, out_channels= 64, kernel_size=(20,2), padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=(10,2), stride=2)

        self.fc1 = nn.Linear(in_features=64*(13)*(2), out_features=500)
        self.relu3 = nn.ReLU()

        self.fc2 = nn.Linear(in_features=500, out_features = 3)

    def forward(self,x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.relu2(x)
        x = self.pool2(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.relu3(x)
        # pass the output to our softmax classifier to get our output
        # predictions
        x = self.fc2(x)
        # return the output predictions
        return x

In [57]:
model = MultiChannelCNN().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-4)
lossFn =nn.CrossEntropyLoss()


In [58]:
num_epochs = 60
for epoch in range(num_epochs):  # loop over the dataset multiple times
    running_loss = 0.0
    for i, data in enumerate(loader, 0):
        inputs, labels = data
        labels = labels.to(device)
        inputs = inputs.to(device)
        optimizer.zero_grad()
        outputs = model(inputs).to(device)
        # _, preds = torch.max(outputs,1)
        loss = lossFn(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {running_loss / len(loader)}")

Epoch 1, Loss: 63042.1103515625
Epoch 2, Loss: 50504.274739583336
Epoch 3, Loss: 28902.653483072918
Epoch 4, Loss: 6060.384826660156
Epoch 5, Loss: 6702.554829915364
Epoch 6, Loss: 3290.701690673828
Epoch 7, Loss: 2043.847132364909
Epoch 8, Loss: 2172.226763407389
Epoch 9, Loss: 1219.530253092448
Epoch 10, Loss: 952.273562113444
Epoch 11, Loss: 316.42119216918945
Epoch 12, Loss: 159.96730740865073
Epoch 13, Loss: 164.1753247578939
Epoch 14, Loss: 248.07035954793295
Epoch 15, Loss: 58.07745488484701
Epoch 16, Loss: 227.9706573486328
Epoch 17, Loss: 307.44896443684894
Epoch 18, Loss: 175.8828010559082
Epoch 19, Loss: 26.81922785441081
Epoch 20, Loss: 0.0
Epoch 21, Loss: 3.186099052429199
Epoch 22, Loss: 21.92144775390625
Epoch 23, Loss: 374.8085352579753
Epoch 24, Loss: 388.85475158691406
Epoch 25, Loss: 0.0
Epoch 26, Loss: 148.32023111979166
Epoch 27, Loss: 245.68664169311523
Epoch 28, Loss: 311.5575935045878
Epoch 29, Loss: 7.256991068522136
Epoch 30, Loss: 105.25927797953288
Epoch 31,

# Test

In [59]:
test =  loadmat("exampleEMGdata120trial_test.mat")
test_trials = test['dataChTimeTr']
test_labels = test['labels'].reshape(-1).tolist()
test_labels = [n-1 for n in test_labels]


In [60]:
test_images = []
for i in range(test_trials.shape[-1]):
    time_domain_data = test_trials[:,:,i]
    spectrograms = []
    for j in range(4):
        f, t, Sxx = signal.spectrogram(time_domain_data[j], fs= sr)
        spectrograms.append(Sxx)
    multichannel_single_trial = np.stack(spectrograms, axis=-1)
    test_images.append(multichannel_single_trial)
test_images = np.array(test_images)

In [61]:
test_images_torch = torch.tensor(test_images.transpose(0,3,1,2), dtype=torch.float32)
test_dataset = TensorDataset(test_images_torch, torch.tensor(test_labels,dtype=torch.long))
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

In [62]:
all_labels = []
all_preds = []

with torch.no_grad():  # Disable gradient computation
    for inputs, labels in test_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, preds = torch.max(outputs,1)
        all_labels.append(labels)
        all_preds.append(preds)

# # Concatenate all the labels and predictions
all_labels = torch.cat(all_labels)
all_preds = torch.cat(all_preds)


In [63]:
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix

# Convert to numpy arrays for use with sklearn
all_labels = all_labels.cpu().numpy()
all_preds = all_preds.cpu().numpy()

# Calculate accuracy
accuracy = accuracy_score(all_labels, all_preds)
print(f'Accuracy: {accuracy * 100:.2f}%')

cm = confusion_matrix(all_labels, all_preds)
print(cm)


Accuracy: 95.00%
[[38  1  1]
 [ 2 38  0]
 [ 2  0 38]]


In [64]:
df = pd.DataFrame(np.stack([all_labels, all_preds]))

In [17]:
df

,0,1,2,3,4,5,6,7,8,9,...,110,111,112,113,114,115,116,117,118,119
0,0,1,0,2,2,0,1,1,1,2,...,2,1,1,2,0,1,2,1,1,1
1,0,1,0,2,2,1,1,1,1,2,...,2,1,1,0,1,1,2,1,1,1
